In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt

Data Analysis

In [2]:
#importing the data of systems 1 through 4 for Part 1 a
os.chdir('/Users/francodelbalso/Downloads')
systems = np.genfromtxt('FinalSystems1-4.csv', delimiter=',')

#seperating the data into each system
system1 = []
system2 = []
system3 = []
system4 = []

for i in range(len(systems)):
    if i < 6:
        system1.append(systems[i])
    elif i < 12: 
        system2.append(systems[i])
    elif i < 18:
        system3.append(systems[i])
    else:
        system4.append(systems[i])

system1 = np.transpose(system1)
system2 = np.transpose(system2)
system3 = np.transpose(system3)
system4 = np.transpose(system4)

print(system1)
print(system2)
print(system3)
print(system4)

[[  2.   3.   4.   5.   6.   7.]
 [  9.  45.  82. 119. 156. 191.]]
[[  2.   3.   4.   5.   6.   7.]
 [ 17.  89. 164. 239. 314. 389.]]
[[  1.   2.   3.   4.   5.   6.]
 [ 55. 122. 191. 259. 326. 393.]]
[[  1.   2.   3.   4.   5.   6.]
 [ 11.  45.  80. 114. 148. 183.]]


In [3]:
#defining quantities
#masses in kg lengths in m
mass25 = 23.4373/1000
mass50 = 50.1393/1000
SpulleyM = 10.0231/1000
SpulleyR = (24.99/2)/1000
LpulleyM = 36.0243/1000
LpulleyR = (50/2)/1000
LowSteelM = 1346.7/1000
LowSteelR = (126.32/2)/1000
UpSteelM = 1354.8/1000
UpSteelR = (126.56/2)/1000
UpAlumM = 470.8/1000
UpAlumR = (126.25/2)/1000
washerM = 1.0306/1000
cylM = 25.48/1000
cylRout = (22.93/2)/1000
cylRin = (15.78/2)/1000
spindleM = 15.83/1000
g_montreal = 9.789

5.1

In [4]:
#calculating the angular acceleration
#here, 6 values will be used, but only 5 acceleratons will be calculated

def AverageAngularAcc(system, Rdisk):
    sum = 0
    for i in range(len(system[0])):
        if i == 0:
            continue #only need first value as the initial vprev
        
        dt = 1 #we are taking 1 second timesteps
        v = system[1][i]*0.002 #tangential velocity as defined in manual
        vprev = system[1][i-1]*0.002
            
        dw = (v-vprev)/Rdisk #change in angular velocity
        
        a = dw/dt
        sum += a
    return sum/(len(system[0])-1) #average of the 5 calculated accels

a1 = AverageAngularAcc(system1,UpSteelR)
a2 = AverageAngularAcc(system2,UpSteelR)
a3 = AverageAngularAcc(system3,UpSteelR)
a4 = AverageAngularAcc(system4,UpSteelR)

#calculating moments of inertia
#will be adding I of pulley to I of disk since they both make up the rotating system
#will be using Icylinder = (MR**2)/2

I1 = (UpSteelM*UpSteelR**2)/2 + (SpulleyM*SpulleyR**2)/2
I2 = (UpSteelM*UpSteelR**2)/2 + (SpulleyM*SpulleyR**2)/2
I3 = (UpSteelM*UpSteelR**2)/2 + (LpulleyM*LpulleyR**2)/2
I4 = (2*UpSteelM*UpSteelR**2)/2 + (LpulleyM*LpulleyR**2)/2

#the torque Ia will be determined for each system

Ia1 = I1*a1
Ia2 = I2*a2
Ia3 = I3*a3
Ia4 = I4*a4

#Now determining the torque by the Fr = mgr method
#m is mass, g is gravitational acceleration, r is radius of pulley

Fr1 = mass25*g_montreal*SpulleyR
Fr2 = mass50*g_montreal*SpulleyR
Fr3 = mass25*g_montreal*LpulleyR
Fr4 = mass25*g_montreal*LpulleyR

In [5]:
#Comparing the torques by %difference

#system1
percentDif1 = 100*abs(Fr1-Ia1)/np.mean([Fr1,Ia1])
print(percentDif1)

#system2
percentDif2 = 100*abs(Fr2-Ia2)/np.mean([Fr2,Ia2])
print(percentDif2)

#system3
percentDif3 = 100*abs(Fr3-Ia3)/np.mean([Fr3,Ia3])
print(percentDif3)

#system4
percentDif4 = 100*abs(Fr4-Ia4)/np.mean([Fr4,Ia4])
print(percentDif4)

8.511226878886534
3.9569346529578433
1.450858902036773
3.0033693343403365


5.1.1

In [6]:
#importing time-distance measurements
distanceTime = np.genfromtxt('distanceTimes.csv', delimiter=',')
distanceTime = np.transpose(distanceTime)
print(distanceTime)

[[15.   30.   45.   60.   75.   90.  ]
 [ 4.48  6.14  7.68  8.76  9.82 10.77]]


In [29]:
#decrease in potential energy
def dU(measures):
    dUs = []
    for i in range(len(measures[0])):
        dUs.append(mass25*g_montreal*(measures[0][i]/100))
    return dUs


#kinetic energy gained by the mass
def KEmass(measures):
    dKmasses = []
    finalVs = []
    for i in range(len(measures[0])):
        dh = measures[0][i]/100 #cm to meters 
        t = measures[1][i] 
        A = 2*dh/t**2 #a as defined in manual
        v = np.sqrt(2*A*dh) #final velocity
        finalVs.append(v)
        dKmasses.append((mass25*v**2)/2) #formula for kinetic energy
    return finalVs, dKmasses


#rotational energy gained by rotating disk
def REdisk(velocities):
    REs = []
    I = (UpSteelM*UpSteelR**2)/2 + (SpulleyM*SpulleyR**2)/2
    for i in range(len(velocities)):
        w = velocities[i]/SpulleyR #final angular velocity from the final velocity of mass and v = rw
        REs.append((I*w**2)/2) #formula for rotational kinetic energy
    return REs


#getting total energy gained
Egained = []
for i in range(len(kineticsMass)):
    Egained.append(kineticsMass[i]+rotEnergies[i])

    
#percent difference between E gained and potential lost
def percentDiff(U,E):
    percents = []
    for i in range(len(U)):
        diff = 100*abs(U[i]-E[i])/np.mean([U[i],E[i]]) #percent difference formula
        percents.append(diff)
    return percents


potentials = dU(distanceTime)
velocities, kineticsMass = KEmass(distanceTime) 
rotEnergies = REdisk(velocities)
Egained = Egained
pctDiffs = percentDiff(potentials,Egained)

print(potentials)
print(Egained)
print(pctDiffs)
    


[0.034414159454999996, 0.06882831890999999, 0.103242478365, 0.13765663781999998, 0.172070797275, 0.20648495673]
[0.03901868577546043, 0.08309060371863901, 0.1194947251873476, 0.16328279872975984, 0.20302321392701034, 0.24305231368093083]
[12.540781460965224, 18.77617950662107, 14.593203616770746, 17.030776161184587, 16.503818097666525, 16.268887746506042]
